<a href="https://colab.research.google.com/github/acewolfag/facenet/blob/main/FaceNet11-14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install facenet-pytorch albumentations opencv-python-headless torch torchvision
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!pip install tqdm
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2
%cd /content
!git clone https://github.com/acewolfag/DatasetDA2.git

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import os

# Định nghĩa các phép biến đổi tăng cường dữ liệu
augmentation = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Rotate(limit=15, p=0.3),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=10, p=0.3),
    A.GaussianBlur(p=0.1),
    A.RandomGamma(p=0.2),
    A.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.2, p=0.2),
    A.Resize(160, 160),
    ToTensorV2()
])

# Dataset tùy chỉnh với tăng cường dữ liệu
class FaceSpoofDataset(Dataset):
    def __init__(self, root_dir, transform=None, augmentations=None):
        self.root_dir = root_dir
        self.transform = transform
        self.augmentations = augmentations
        self.data = []
        self.labels = []

        # Tải dữ liệu từ thư mục
        for label, folder in enumerate(['Real', 'Fake']):
            folder_path = os.path.join(root_dir, folder)
            for person in os.listdir(folder_path):
                person_path = os.path.join(folder_path, person)
                for img_name in os.listdir(person_path):
                    img_path = os.path.join(person_path, img_name)
                    self.data.append(img_path)
                    self.labels.append(label)  # 0 = real, 1 = fake

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data[idx]
        label = self.labels[idx]
        image = np.array(Image.open(img_path).convert("RGB"))

        if self.augmentations:
            image = self.augmentations(image=image)["image"]

        return image, label


In [ ]:
from facenet_pytorch import InceptionResnetV1
import torch.nn as nn
import torch.optim as optim

# Khởi tạo mô hình FaceNet và classifier
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = InceptionResnetV1(pretrained='vggface2').to(device).eval()  # FaceNet
classifier = nn.Sequential(
    nn.Linear(512, 128),
    nn.ReLU(),
    nn.Linear(128, 2)
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=0.001)


In [ ]:
from tqdm import tqdm

# Khởi tạo DataLoader
dataset = FaceSpoofDataset(root_dir="/content/DatasetDA2/", augmentations=augmentation)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Huấn luyện model
epochs = 10
for epoch in range(epochs):
    # Thêm thanh tiến trình vào dataloader
    progress_bar = tqdm(dataloader, desc=f"Epoch [{epoch+1}/{epochs}]", leave=False)

    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)

        # Đảm bảo images có kiểu FloatTensor
        images = images.float()

        # Trích xuất đặc trưng từ FaceNet
        with torch.no_grad():
            embeddings = model(images)

        # Phân loại với classifier
        outputs = classifier(embeddings)
        loss = criterion(outputs, labels)

        # Tối ưu
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Cập nhật thanh tiến trình với giá trị loss
        progress_bar.set_postfix({"Loss": loss.item()})

    # In kết quả sau mỗi epoch
    print(f"Epoch [{epoch+1}/{epochs}] completed, Loss: {loss.item():.4f}")


In [ ]:
torch.save(classifier.state_dict(), 'spoof_classifier.pth')


In [ ]:
from albumentations import Compose, Resize, Normalize
from albumentations.pytorch import ToTensorV2

# Định nghĩa transform để tiền xử lý ảnh
transform = Compose([
    Resize(160, 160),
    Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
    ToTensorV2()
])

In [ ]:
# Tải lại mô hình và lớp phân loại
classifier.load_state_dict(torch.load('spoof_classifier.pth'))
classifier.eval()

# Hàm tiền xử lý ảnh
def preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image = np.array(image)
    image = transform(image=image)["image"]
    return image.float().unsqueeze(0).to(device)  # Chuyển ảnh sang FloatTensor và thêm chiều batch


# Hàm phát hiện giả mạo
def detect_spoof(image_path):
    image_tensor = preprocess_image(image_path)

    with torch.no_grad():
        embedding = model(image_tensor)
        output = classifier(embedding)
        _, predicted = torch.max(output, 1)

    if predicted.item() == 0:
        print("Real person detected!")
    else:
        print("Spoof detected!")


In [ ]:
test_image_path = '/content/DatasetDA2/Real/Huy/frame_107.jpg'
detect_spoof(test_image_path)

In [ ]:
!pip install onnx
!pip install onnxruntime


In [ ]:
import torch
import torch.nn as nn

# Tạo lại kiến trúc của classifier
classifier = nn.Sequential(
    nn.Linear(512, 128),
    nn.ReLU(),
    nn.Linear(128, 2)
)

# Tải trọng số vào mô hình
classifier.load_state_dict(torch.load('spoof_classifier.pth'))
classifier.eval()  # Chuyển sang chế độ đánh giá

# Chuyển đổi sang ONNX
dummy_input = torch.randn(1, 512)  # Giả sử đầu vào của bạn có kích thước [1, 512]
torch.onnx.export(classifier, dummy_input, "spoof_classifier.onnx",
                  input_names=['input'], output_names=['output'],
                  dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}})


In [1]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import cv2
import numpy as np
from PIL import Image
import io
from facenet_pytorch import MTCNN
import torch

# Initialize MTCNN for face detection
mtcnn = MTCNN(keep_all=True)

# Load the PyTorch model
model_path = '/content/spoof_classifier.pth'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.load(model_path, map_location=device)
model.eval()

# Define class labels, e.g., Real vs. Fake
classes = ['Real', 'Fake']

# JavaScript to display video stream
def video_stream():
    js = """
    const video = document.createElement('video');
    video.setAttribute('playsinline', '');
    video.style.display = 'block';
    document.body.appendChild(video);

    navigator.mediaDevices.getUserMedia({video: true}).then(stream => {
      video.srcObject = stream;
      video.play();
    });

    const canvas = document.createElement('canvas');
    canvas.width = 640;
    canvas.height = 480;
    const context = canvas.getContext('2d');

    function captureFrame() {
      context.drawImage(video, 0, 0, canvas.width, canvas.height);
      return canvas.toDataURL('image/jpeg', 0.8);
    }

    async function stream() {
      while (true) {
        const frame = captureFrame();
        google.colab.kernel.invokeFunction('notebook.captureFrame', [frame], {});
        await new Promise(resolve => setTimeout(resolve, 100));
      }
    }

    stream();
    """
    display(Javascript(js))

# Convert JavaScript image data to OpenCV format
def js_to_image(js_reply):
    image_bytes = b64decode(js_reply.split(',')[1])
    image = Image.open(io.BytesIO(image_bytes))
    return cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

# Register callback function to handle each frame
from google.colab import output
def capture_frame(js_reply):
    frame = js_to_image(js_reply)

    # Detect faces and get bounding boxes
    boxes, _ = mtcnn.detect(frame)

    if boxes is not None:
        for box in boxes:
            # Extract and preprocess the face
            x1, y1, x2, y2 = map(int, box)
            face = frame[y1:y2, x1:x2]
            face_resized = cv2.resize(face, (128, 128))  # Assuming model input size is 128x128
            face_tensor = torch.from_numpy(face_resized).permute(2, 0, 1).unsqueeze(0).float().to(device)

            # Run the PyTorch model for real vs. fake detection
            with torch.no_grad():
                pred = model(face_tensor)
                class_id = torch.argmax(pred, dim=1).item()
                class_name = classes[class_id]

            # Draw bounding box and label
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{class_name}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    # Display the frame with overlays
    cv2_imshow(frame)

output.register_callback('notebook.captureFrame', capture_frame)

# Start video stream
video_stream()


ModuleNotFoundError: No module named 'facenet_pytorch'